In [1]:
import warnings

def warn(*args, **kwargs):
    pass
warnings.warn = warn

import pandas as pd
from autosklearn.experimental.askl2 import AutoSklearn2Classifier as AutoClassifier
from tpot import TPOTClassifier

import pickle

In [2]:
df = pd.read_csv('CRT_clean.csv')
df.head()

,CRT,ACT,batball,3flips,widgets,trucks,lilypads,water,giftwrap,Jerry,...,school_Toledo,school_UM (dearborn),Sex_Male,RACE_Asian,RACE_Black,RACE_Latino,RACE_Other,RACE_White,Prestige_low,Prestige_middle
0,0.0,26.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0,0,1,1,0
1,2.0,23.0,1,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0,1,0,0,1,0,0,0,1,0
2,1.0,27.0,0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0,1,1,0,0,0,0,1,1,0
3,0.0,22.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0,1,0,0,0,1,0
4,3.0,31.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,1,1,0,0,0,1,0,1,0


In [5]:
y_var = '3.4K v 3.8K(2)'
X = df.drop(columns=y_var).to_numpy()
y = df[y_var].to_numpy()

In [13]:
askl_clf = AutoClassifier(time_left_for_this_task=120).fit(X, y)
pickle.dump(askl_clf.show_models(), open('askl_clf.p', 'wb'))
print(askl_clf.sprint_statistics())

Using no backup selector
auto-sklearn results:
  Dataset name: ea4d9bd6560e6c8a08315c95c41b2768
  Metric: accuracy
  Best validation score: 0.682089
  Number of target algorithm runs: 20
  Number of successful target algorithm runs: 16
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0



In [17]:
from sklearn.model_selection import KFold

kf = KFold(5)
xval = []
for train_idx, test_idx in kf.split(X, y):
    X_train, X_test, y_train, y_test = \
        X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    askl_clf.refit(X_train, y_train)
    xval.append(askl_clf.score(X_test, y_test))
sum(xval) / len(xval)

0.5708715184186882

In [68]:
clf = TPOTClassifier(max_time_mins=30)
clf.fit(X, y)
clf.export('tpot_pipeline.py')

In [62]:
from sklearn.model_selection import cross_validate

from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator, ZeroCount

pipeline_clf = make_pipeline(
    SelectPercentile(score_func=f_classif, percentile=38),
    StackingEstimator(estimator=KNeighborsClassifier(n_neighbors=15, p=2, weights="uniform")),
    ZeroCount(),
    GaussianNB()
)

cross_validate(pipeline_clf, X, y, cv=5)['test_score'].mean()

/home/dsbowen/crt/venv/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/dsbowen/crt/venv/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


0.6275651392632524

In [67]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from tpot.builtins import OneHotEncoder

pipeline_clf = make_pipeline(
    OneHotEncoder(minimum_fraction=0.25, sparse=False, threshold=10),
    RFE(estimator=ExtraTreesClassifier(criterion="gini", max_features=0.1, n_estimators=100), step=0.45),
    GaussianNB()
)

cross_validate(pipeline_clf, X, y, cv=5)['test_score'].mean()

0.6086612758310872

In [100]:
from sklearn.cluster import FeatureAgglomeration
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

pipeline_clf = make_pipeline(
    FeatureAgglomeration(affinity="euclidean", linkage="ward"),
    StackingEstimator(estimator=SGDClassifier(alpha=0.0, eta0=0.1, fit_intercept=False, l1_ratio=0.5, learning_rate="constant", loss="perceptron", penalty="elasticnet", power_t=0.5)),
    StackingEstimator(estimator=MLPClassifier(alpha=0.001, learning_rate_init=1.0)),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.3, min_samples_leaf=7, min_samples_split=13, n_estimators=100)
)

cross_validate(pipeline_clf, X, y, cv=10)['test_score'].mean()

0.6389695210449927

In [50]:
from sklearn.model_selection import KFold

kf = KFold(10)
xval = []
for train_idx, test_idx in kf.split(X, y):
    X_train, X_test, y_train, y_test = \
        X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    pipeline_clf.fit(X_train, y_train)
    xval.append(pipeline_clf.score(X_test, y_test))
sum(xval) / len(xval)

0.6029390420899855

In [ ]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

paths = ['CRT_clean.csv']
target_name = '3.4K v 3.8K(2)'
data = Reader().clean('CRT_clean.csv')
data = Drift_thresholder().fit_transform(data)
Optimiser.evaluate(None, data)